In [4]:
# !pip install tensorflow-gpu==2.0.0

In [22]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import glob
import os

In [23]:
# os.listdir('../input/cat-and-dog/training_set/training_set/')

In [24]:
print('Tensorflow version: {}'.format(tf.__version__))

Tensorflow version: 2.1.0


In [25]:
# 本地图片的位置
# '../input/dc_2000/train/*/*.jpg'

train_image_path = glob.glob('../input/cat-and-dog/training_set/training_set/*/*.jpg')

len(train_image_path)# kaggle上8005张图片
train_image_path[-5:]

['../input/cat-and-dog/training_set/training_set/cats/cat.3669.jpg',
 '../input/cat-and-dog/training_set/training_set/cats/cat.2859.jpg',
 '../input/cat-and-dog/training_set/training_set/cats/cat.963.jpg',
 '../input/cat-and-dog/training_set/training_set/cats/cat.137.jpg',
 '../input/cat-and-dog/training_set/training_set/cats/cat.3737.jpg']

In [26]:
np.random.shuffle(train_image_path) # 打乱路径, 前面几千张全是狗， 后面的全是猫

In [27]:
# 本地数据集读取标签
# train_image_label = [int(path.split('input/dc_2000/train/')[0].split('\\')[1].split('.')[0] == 'cat') for path in train_image_path]

# kaggle上的数据集读取标签
# ../input/cat-and-dog/training_set/training_set/cats/cat.2853.jpg
train_image_label = [int(path.split('/training_set/training_set/')[1].split('/')[1].split('.')[0] == 'cat') for path in train_image_path]
train_image_label[-5:]

[1, 1, 0, 1, 0]

In [28]:
# 查看读取到的图片是否正确
# image = tf.io.read_file(train_image_path[0])
# image = tf.image.decode_jpeg(image, channels=3)
# print(image.shape)
# plt.imshow(image)

In [29]:
def load_preprocess_image(path, label):
    '''
    func:根据路径载入图像和label
    '''


    image = tf.io.read_file(path)   # 根据文件路径读取文件
    image = tf.image.decode_jpeg(image,channels=3) # jpg解码
    image = tf.image.resize(image, [256, 256]) # 统一大小
    
#    image = tf.image.random_crop(image, [256, 256, 3])
    # image = tf.image.random_flip_left_right(image)
    # image = tf.image.random_flip_up_down(image)
#    image = tf.image.random_brightness(image, 0.5)
#    image = tf.image.random_contrast(image, 0, 1)
    image = tf.cast(image, tf.float32)  # 转换数据类型
    image = image/255   # 统一数据范围

    label = tf.reshape(label, (1,)) # reshape成(num,)
    return image, label

In [30]:
#tf.image.convert_image_dtype

In [31]:
train_image_ds = tf.data.Dataset.from_tensor_slices((train_image_path, train_image_label)) # load数据集
# train_image_ds

In [32]:
# 根据计算机的特征，自动的使用并行运算
AUTOTUNE = tf.data.experimental.AUTOTUNE
# AUTOTUNE

In [33]:
# 预处理
train_image_ds = train_image_ds.map(load_preprocess_image, num_parallel_calls=AUTOTUNE) # 载入图像
# print(train_image_ds)

In [34]:
# for img, label in train_image_ds.take(1):
#     plt.imshow(img)

In [35]:
# 设定batch
BATCH_SIZE = 32
train_count = len(train_image_path) # 训练数据集的个数

In [36]:
# 设立batch, train_image_ds是一个可迭代对象
train_image_ds = train_image_ds.shuffle(500).batch(BATCH_SIZE)
train_image_ds = train_image_ds.prefetch(AUTOTUNE)

In [37]:
# imgs,labels=next(iter(train_image_ds))
# imgs.shape
# imgs[0]

In [38]:
# train_image_path = glob.glob('../input/dc_2000/train/*/*.jpg')
# train_image_label = [int(path.split('input/dc_2000/train/')[0].split('\\')[1].split('.')[0] == 'cat') for path in train_image_path]
# test_image_path = glob.glob('../input/dc_2000/test/*/*.jpg')
# test_image_label = [int(path.split('input/dc_2000/train/')[0].split('\\')[0].split('.')[0] == 'cat') for path in test_image_path]

# train_image_path = glob.glob('../input/cat-and-dog/training_set/training_set/*/*.jpg')
test_image_path = glob.glob('../input/cat-and-dog/test_set/test_set/*/*.jpg')
# train_image_label = [int(path.split('/training_set/training_set/')[1].split('/')[1].split('.')[0] == 'cat') for path in train_image_path]
test_image_label = [int(path.split('/test_set/test_set/')[1].split('/')[1].split('.')[0] == 'cat') for path in test_image_path]

# 载入测试图片
test_image_ds = tf.data.Dataset.from_tensor_slices((test_image_path, test_image_label))
test_image_ds = test_image_ds.map(load_preprocess_image, num_parallel_calls=AUTOTUNE)
test_image_ds = test_image_ds.batch(BATCH_SIZE)
test_image_ds = test_image_ds.prefetch(AUTOTUNE)

In [39]:
len(test_image_path)

2023

In [56]:
model = keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), input_shape=(256, 256, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    # tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    # tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    # tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    # tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    
#     tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1)
])

In [57]:
# model_test = keras.Sequential([
#     tf.keras.layers.Conv2D(64, (3, 3), input_shape=(256, 256, 3), activation='relu'),
#     tf.keras.layers.BatchNormalization(),
    
#     # tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#     # tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPooling2D(),
    
#     tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
#     tf.keras.layers.BatchNormalization(),
    
#     # tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
#     # tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPooling2D(),
    
#     tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
#     tf.keras.layers.BatchNormalization(),
    
#     # tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
#     # tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPooling2D(),
    
#     tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
#     tf.keras.layers.BatchNormalization(),
    
#     # tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
#     # tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPooling2D(),
    
#     tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     # tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'),
#     # tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.GlobalAveragePooling2D(),
    
#     tf.keras.layers.Dense(256, activation='relu'),
#     # tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dense(1)
# ])

In [58]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 254, 254, 64)      1792      
_________________________________________________________________
batch_normalization_6 (Batch (None, 254, 254, 64)      256       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 125, 125, 128)     73856     
_________________________________________________________________
batch_normalization_7 (Batch (None, 125, 125, 128)     512       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 60, 60, 256)      

In [59]:
# tf.keras.losses.binary_crossentropy([0.,0.,1.,1.], [1.,1.,1.,1.])#二元交叉熵定义loss

In [60]:
# [https://www.jianshu.com/p/aebcaf8af76e]
# 使用Adam优化, 具体一些的对Adam的解释可以看上面的博客
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [61]:
epoch_loss_avg = tf.keras.metrics.Mean('train_loss') # 
train_accuracy = tf.keras.metrics.Accuracy() # 计算正确率

epoch_loss_avg_test = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.Accuracy()

In [62]:
# train_accuracy([1,0,1], [1,1,1])

In [63]:
def train_step(model, images, labels):
    """
    func:训练集求梯度
    """
    with tf.GradientTape() as t:
        pred = model(images) # 使用模型预测结果
        loss_step = tf.keras.losses.BinaryCrossentropy(from_logits=True)(labels, pred) # 计算损失值
    
    grads = t.gradient(loss_step, model.trainable_variables) #计算dloss/d(参数)
    optimizer.apply_gradients(zip(grads, model.trainable_variables)) #根据dloss/d(variable)进行优化

    epoch_loss_avg(loss_step)# 计算loss的均值
    train_accuracy(labels, tf.cast(pred>0, tf.int32))# 计算精确度

In [54]:
def test_step(model, images, labels):
    pred = model(images, training=False)
#     pred = model(images, training=False，dropout=False)
    loss_step = tf.keras.losses.BinaryCrossentropy(from_logits=True)(labels, pred)
    epoch_loss_avg_test(loss_step)
    test_accuracy(labels, tf.cast(pred>0, tf.int32))

In [52]:
train_loss_results = []
train_acc_results = []

test_loss_results = []
test_acc_results = []

In [53]:
num_epochs = 30

In [ ]:
for epoch in range(num_epochs):
#     print(train_image_ds)
    for imgs_, labels_ in train_image_ds:
        train_step(model, imgs_, labels_)
        print('.', end='')
    print()
    
    train_loss_results.append(epoch_loss_avg.result())
    train_acc_results.append(train_accuracy.result())
    
    
    for imgs_, labels_ in test_image_ds:
        test_step(model, imgs_, labels_)
        
    test_loss_results.append(epoch_loss_avg_test.result())
    test_acc_results.append(test_accuracy.result())
    
    print('Epoch:{}: loss: {:.3f}, accuracy: {:.3f}, test_loss: {:.3f}, test_accuracy: {:.3f}'.format(
        epoch + 1,
        epoch_loss_avg.result(),
        train_accuracy.result(),
        epoch_loss_avg_test.result(),
        test_accuracy.result()
    ))
    
    epoch_loss_avg.reset_states()
    train_accuracy.reset_states()
    
    epoch_loss_avg_test.reset_states()
    test_accuracy.reset_states()

...........................................................................................................................................................................................................................................................
Epoch:1: loss: 0.681, accuracy: 0.555, test_loss: 0.661, test_accuracy: 0.599
...........................................................................................................................................................................................................................................................
Epoch:2: loss: 0.633, accuracy: 0.635, test_loss: 0.660, test_accuracy: 0.589
...........................................................................................................................................................................................................................................................
Epoch:3: loss: 0.584, accuracy: 0.702, test_loss: 0.579, test_accuracy: 0.697
...........................................................................................................................................................................................................................................................
Epoch:4: loss: 0.550, accuracy: 0.722, test_loss: 0.547, test_accuracy: 0.722
...........................................................................................................................................................................................................................................................
Epoch:5: loss: 0.526, accuracy: 0.739, test_loss: 0.564, test_accuracy: 0.709
...........................................................................................................................................................................................................................................................
Epoch:6: loss: 0.500, accuracy: 0.757, test_loss: 0.489, test_accuracy: 0.773
...........................................................................................................................................................................................................................................................
Epoch:7: loss: 0.479, accuracy: 0.776, test_loss: 0.512, test_accuracy: 0.757
...........................................................................................................................................................................................................................................................
Epoch:8: loss: 0.461, accuracy: 0.783, test_loss: 0.568, test_accuracy: 0.712
...........................................................................................................................................................................................................................................................
Epoch:9: loss: 0.436, accuracy: 0.801, test_loss: 0.429, test_accuracy: 0.817
...........................................................................................................................................................................................................................................................
Epoch:10: loss: 0.408, accuracy: 0.815, test_loss: 0.456, test_accuracy: 0.789
...........................................................................................................................................................................................................................................................
Epoch:11: loss: 0.380, accuracy: 0.831, test_loss: 0.402, test_accuracy: 0.827
...........................................................................................................................................................................................................................................................
Epoch:12: loss: 0.357, accuracy: 0.844, test_loss: 0.426, test_accuracy: 0.812
...........................................................................................................................................................................................................................................................
Epoch:13: loss: 0.346, accuracy: 0.850, test_loss: 0.445, test_accuracy: 0.799
...........................................................................................................................................................................................................................................................
Epoch:14: loss: 0.314, accuracy: 0.865, test_loss: 0.383, test_accuracy: 0.826
...........................................................................................................................................................................................................................................................
Epoch:15: loss: 0.292, accuracy: 0.873, test_loss: 0.409, test_accuracy: 0.826
...........................................................................................................................................................................................................................................................
Epoch:16: loss: 0.281, accuracy: 0.880, test_loss: 0.335, test_accuracy: 0.852
...........................................................................................................................................................................................................................................................
Epoch:17: loss: 0.244, accuracy: 0.895, test_loss: 0.358, test_accuracy: 0.846
...........................................................................................................................................................................................................................................................
Epoch:18: loss: 0.236, accuracy: 0.901, test_loss: 0.328, test_accuracy: 0.865
...........................................................................................................................................................................................................................................................
Epoch:19: loss: 0.219, accuracy: 0.909, test_loss: 0.358, test_accuracy: 0.863
...........................................................................................................................................................................................................................................................
Epoch:20: loss: 0.197, accuracy: 0.919, test_loss: 0.323, test_accuracy: 0.876
...........................................................................................................................................................................................................................................................
Epoch:21: loss: 0.177, accuracy: 0.928, test_loss: 0.293, test_accuracy: 0.882
...........................................................................................................................................................................................................................................................
Epoch:22: loss: 0.158, accuracy: 0.938, test_loss: 0.302, test_accuracy: 0.886
...........................................................................................................................................................................................................................................................
Epoch:23: loss: 0.148, accuracy: 0.941, test_loss: 0.296, test_accuracy: 0.892
...........................................................................................................................................................................................................................................................
Epoch:24: loss: 0.128, accuracy: 0.951, test_loss: 0.361, test_accuracy: 0.866
...........................................................................................................................................................................................................................................................
Epoch:25: loss: 0.112, accuracy: 0.958, test_loss: 0.328, test_accuracy: 0.871
...........................................................................................................................................................................................................................................................
Epoch:26: loss: 0.124, accuracy: 0.953, test_loss: 0.333, test_accuracy: 0.886
...........................................................................................................................................................................................................................................................
Epoch:27: loss: 0.094, accuracy: 0.966, test_loss: 0.355, test_accuracy: 0.881
...........................................................................................................................................................................................................................................................
Epoch:28: loss: 0.073, accuracy: 0.973, test_loss: 0.355, test_accuracy: 0.885
...........................................................................................................................................................................................................................................................
Epoch:29: loss: 0.086, accuracy: 0.967, test_loss: 0.318, test_accuracy: 0.888
...........................................................................................................................................................................................................................................................
Epoch:30: loss: 0.070, accuracy: 0.975, test_loss: 0.384, test_accuracy: 0.881

In [ ]:
print(len(train_acc_results))
print(len(test_acc_results))
plt.plot(range(1,num_epochs+1),train_acc_results,label='train_accuracy')
plt.plot(range(1,num_epochs+1),test_acc_results,label='test_accuracy')
plt.legend()

In [ ]:
plt.plot(range(1,num_epochs+1),train_loss_results,label='train_loss')
plt.plot(range(1,num_epochs+1),test_loss_results,label='test_loss')
plt.legend()